# Análise Exploratória - Camada SILVER

## Introdução

Este notebook inicia a análise exploratória dos dados da camada **SILVER** do projeto "Airline On-Time / Delay Causes".

### O que é a camada Silver?

No contexto da arquitetura de **Medalhão** (Medallion Architecture):

- **RAW (Bronze)**: Dados brutos, sem tratamento
- **SILVER**: Dados limpos, padronizados e validados (onde estamos agora)
- **GOLD**: Dados agregados e modelados para dashboards e relatórios (próximo passo)

A camada SILVER contém dados que já passaram por:
- Limpeza e padronização de colunas
- Remoção de duplicados e colunas vazias
- Conversão de tipos de dados
- Tratamento de valores nulos e negativos
- Detecção de outliers (com flags)
- Normalização de campos categóricos

### Objetivo deste notebook

Este notebook valida e explora a tabela `silver.silver_airline_on_time` para:
1. Verificar a qualidade dos dados após o ETL
2. Realizar análises exploratórias iniciais
3. Identificar padrões e insights preliminares
4. Preparar o terreno para análises mais profundas e modelagem na camada GOLD


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sqlalchemy import create_engine, text
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)


## 1. Conexão com PostgreSQL

Conectamos ao banco de dados PostgreSQL para acessar a tabela `silver.silver_airline_on_time`.


In [ ]:
POSTGRES_DB = os.getenv('POSTGRES_DB', 'airline_delay_causes')
POSTGRES_USER = os.getenv('POSTGRES_USER', 'postgres')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD', 'postgres')
POSTGRES_HOST = 'localhost'
POSTGRES_PORT = 5432

connection_string = f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"

print("=" * 80)
print("Testando conexão com PostgreSQL")
print("=" * 80)
print(f"Host: {POSTGRES_HOST}:{POSTGRES_PORT}")
print(f"Database: {POSTGRES_DB}")
print(f"User: {POSTGRES_USER}")

try:
    engine = create_engine(connection_string, echo=False)
    with engine.connect() as conn:
        result = conn.execute(text("SELECT 1 as test;"))
        test_value = result.fetchone()[0]
        if test_value == 1:
            print("\nConexão estabelecida com sucesso!")
        else:
            print("\nConexão estabelecida, mas teste retornou valor inesperado.")
except Exception as e:
    print(f"\nErro ao conectar ao PostgreSQL: {e}")
    print("\nPossíveis causas:")
    print("  1. Container Docker não está rodando (execute: docker-compose up -d)")
    print("  2. Credenciais incorretas (verifique variáveis de ambiente)")
    print("  3. Porta 5432 não está acessível")
    print("  4. Banco de dados não existe")
    raise


## 2. Carga da Tabela Silver

Carregamos a tabela `silver.silver_airline_on_time` completa para um DataFrame do Pandas.


In [ ]:
print("=" * 80)
print("Carregando dados da tabela silver.silver_airline_on_time")
print("=" * 80)

query = "SELECT * FROM silver.silver_airline_on_time;"
df_silver = pd.read_sql(query, con=engine)

print(f"\nShape: {df_silver.shape[0]:,} linhas × {df_silver.shape[1]} colunas")
print(f"\nPrimeiras 5 linhas:")
display(df_silver.head(5))
print(f"\nInformações do DataFrame:")
df_silver.info()


## 3. Validações Rápidas Pós-ETL

Realizamos verificações de qualidade dos dados para garantir que o ETL funcionou corretamente.


In [ ]:
print("=" * 80)
print("3.1 Contagem de Nulos por Coluna (Top 15)")
print("=" * 80)

null_counts = df_silver.isnull().sum().sort_values(ascending=False)
null_counts = null_counts[null_counts > 0].head(15)

if len(null_counts) > 0:
    null_df = pd.DataFrame({
        'Coluna': null_counts.index,
        'Nulos': null_counts.values,
        'Percentual': (null_counts.values / len(df_silver) * 100).round(2)
    })
    display(null_df)
else:
    print("Nenhum valor nulo encontrado nas colunas principais.")


In [ ]:
print("=" * 80)
print("3.2 Verificação de Duplicados")
print("=" * 80)

total_duplicates = df_silver.duplicated().sum()
print(f"Total de linhas duplicadas: {total_duplicates:,}")

if total_duplicates > 0:
    print(f"Percentual de duplicados: {(total_duplicates / len(df_silver) * 100):.2f}%")
    print("\nExemplo de linhas duplicadas:")
    display(df_silver[df_silver.duplicated(keep=False)].head(10))
else:
    print("Nenhum duplicado encontrado.")


In [ ]:
print("=" * 80)
print("3.3 Estatística Descritiva (Colunas Numéricas)")
print("=" * 80)

numeric_cols = df_silver.select_dtypes(include=[np.number]).columns.tolist()
if len(numeric_cols) > 0:
    display(df_silver[numeric_cols].describe())
else:
    print("Nenhuma coluna numérica encontrada.")


In [ ]:
print("=" * 80)
print("3.4 Verificação de Colunas Suspeitas")
print("=" * 80)

suspicious_cols = []

for col in df_silver.columns:
    if col.lower().startswith('unnamed'):
        suspicious_cols.append((col, 'Coluna com nome "unnamed"'))
    
    null_pct = (df_silver[col].isnull().sum() / len(df_silver)) * 100
    if null_pct == 100:
        suspicious_cols.append((col, 'Coluna 100% nula'))

if len(suspicious_cols) > 0:
    print("Colunas suspeitas encontradas:")
    for col, reason in suspicious_cols:
        print(f"  - {col}: {reason}")
else:
    print("Nenhuma coluna suspeita encontrada.")


## 4. Análises Iniciais

Realizamos análises exploratórias básicas para entender os dados e identificar padrões iniciais.


In [ ]:
print("=" * 80)
print("4.1 Heatmap de Correlação (Colunas Numéricas)")
print("=" * 80)

numeric_cols = df_silver.select_dtypes(include=[np.number]).columns.tolist()

if len(numeric_cols) >= 2:
    corr_matrix = df_silver[numeric_cols].corr()
    
    plt.figure(figsize=(14, 10))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
    plt.title('Matriz de Correlação - Colunas Numéricas', fontsize=16, pad=20)
    plt.tight_layout()
    plt.show()
else:
    print(f"É necessário pelo menos 2 colunas numéricas. Encontradas: {len(numeric_cols)}")


In [ ]:
print("=" * 80)
print("4.2 Distribuição da Métrica Principal (Maior Variância)")
print("=" * 80)

numeric_cols = df_silver.select_dtypes(include=[np.number]).columns.tolist()

if len(numeric_cols) > 0:
    variances = df_silver[numeric_cols].var().sort_values(ascending=False)
    main_metric = variances.index[0]
    
    print(f"Métrica selecionada: {main_metric} (variância: {variances.iloc[0]:,.2f})")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    axes[0].hist(df_silver[main_metric].dropna(), bins=50, edgecolor='black', alpha=0.7)
    axes[0].set_title(f'Histograma - {main_metric}', fontsize=12)
    axes[0].set_xlabel(main_metric)
    axes[0].set_ylabel('Frequência')
    axes[0].grid(True, alpha=0.3)
    
    axes[1].boxplot(df_silver[main_metric].dropna(), vert=True)
    axes[1].set_title(f'Boxplot - {main_metric}', fontsize=12)
    axes[1].set_ylabel(main_metric)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Nenhuma coluna numérica encontrada.")


In [ ]:
print("=" * 80)
print("4.3 Top 10 Categóricas Mais Frequentes")
print("=" * 80)

categorical_cols = df_silver.select_dtypes(include=['object', 'string']).columns.tolist()

if len(categorical_cols) > 0:
    cardinalities = {}
    for col in categorical_cols:
        unique_count = df_silver[col].nunique()
        if 2 <= unique_count <= 100:
            cardinalities[col] = unique_count
    
    if len(cardinalities) > 0:
        selected_col = max(cardinalities, key=cardinalities.get)
        top_values = df_silver[selected_col].value_counts().head(10)
        
        print(f"Coluna selecionada: {selected_col} ({cardinalities[selected_col]} valores únicos)")
        
        plt.figure(figsize=(12, 6))
        top_values.plot(kind='bar', color='steelblue', edgecolor='black', alpha=0.7)
        plt.title(f'Top 10 - {selected_col}', fontsize=14, pad=15)
        plt.xlabel(selected_col)
        plt.ylabel('Frequência')
        plt.xticks(rotation=45, ha='right')
        plt.grid(True, alpha=0.3, axis='y')
        plt.tight_layout()
        plt.show()
        
        print("\nValores:")
        display(pd.DataFrame({
            selected_col: top_values.index,
            'Frequência': top_values.values,
            'Percentual': (top_values.values / len(df_silver) * 100).round(2)
        }))
    else:
        print("Nenhuma coluna categórica com cardinalidade adequada encontrada.")
else:
    print("Nenhuma coluna categórica encontrada.")


## 5. Próximos Passos

Este notebook serve como base inicial para a análise. Os próximos passos recomendados são:

### Análises Temporais
1. **Análise de tendências ao longo do tempo**: Criar gráficos de linha mostrando evolução de métricas principais (arr_delay, arr_flights) por ano/mês
2. **Sazonalidade**: Identificar padrões sazonais (meses com mais atrasos, anos com pior performance)
3. **Comparação ano a ano**: Comparar métricas entre diferentes anos para identificar melhorias ou deterioração

### Análises por Companhia Aérea e Aeroporto
4. **Top companhias aéreas com mais atrasos**: Ranking de carriers por total de atrasos, taxa de atraso, etc.
5. **Top aeroportos com mais problemas**: Identificar aeroportos com maior número de atrasos, cancelamentos ou desvios
6. **Análise de causas de atraso**: Comparar as diferentes causas (carrier, weather, NAS, security, late_aircraft) por carrier/airport

### Métricas Derivadas e KPIs
7. **Taxa de atraso**: Calcular percentual de voos com atraso >= 15 minutos (arr_del15 / arr_flights)
8. **Tempo médio de atraso**: Calcular média de minutos de atraso por carrier, airport, mês
9. **Taxa de cancelamento/desvio**: Calcular percentuais de voos cancelados e desviados

### Preparação para Camada GOLD
10. **Definir perguntas de negócio**: Listar as principais perguntas que o DW deve responder
11. **Identificar dimensões e fatos**: Preparar estrutura dimensional (tempo, carrier, airport) e fatos (delays, flights)
12. **Agregações necessárias**: Definir quais agregações serão necessárias na camada GOLD (ex: total de atrasos por mês/carrier)

### Visualizações Avançadas
13. **Mapas geográficos**: Se houver dados de localização, criar mapas de calor por aeroporto
14. **Dashboards interativos**: Preparar visualizações que permitam drill-down por múltiplas dimensões
15. **Análise de outliers**: Investigar mais profundamente os registros marcados como outliers (is_outlier_arr_delay = 1)

### Qualidade de Dados
16. **Validações adicionais**: Verificar consistências entre colunas (ex: soma de delays por causa deve ser próxima de arr_delay)
17. **Análise de completude**: Verificar se há períodos com dados faltantes ou incompletos
